In [13]:
from sklearn.model_selection import train_test_split
import DeepMIMOv3 as DeepMIMO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from noise import *
from sklearn.preprocessing import MinMaxScaler

In [14]:
def create_dataset(scenario_name, 
                   n_bs_y, n_bs_x, n_ue_y, n_ue_x, 
                   n_paths, n_subcarriers):
    # Load the default params, set constant params
    params = DeepMIMO.default_params()
    params['dataset_folder'] = r'scenarios'
    
    # Set params
    # Which scenario
    params['scenario'] = scenario_name
    # For the antennas
    params['bs_antenna']['shape'] = np.array([n_bs_y, n_bs_x])
    params['ue_antenna']['shape'] = np.array([n_ue_y, n_ue_x])
    # For the path
    params['num_paths'] = n_paths
    params['OFDM']['subcarriers'] = n_subcarriers
    
    # Create dataset
    dataset = DeepMIMO.generate_data(params)
    
    return dataset

In [15]:
def generate_H(dataset):
    i = 0  # Only one base station in scenario I2
    j = 0  # Choose the first user
    H = dataset[i]['user']['channel'][j]
    H = H[:, :, 0]  # Choose the first subcarrier
    return H

def generate_H_abs(dataset):
    H = generate_H(dataset)
    H_abs = np.abs(H)

    return H_abs

In [16]:
# Noise functions
def gaussian(H_true, std):
    return np.random.normal(0, std, H_true.shape)

def pink(H_true, amplitude):
    return amplitude*pink_noise(H_true)

In [17]:
# SNR
def calculate_snr(y_true, y_noise): ### write own code later
    # Calculate signal power
    signal_power = np.mean(y_true ** 2)
    # Calculate noise power
    noise_power = np.mean((y_noise - y_true) ** 2)
    # Calculate (linear) SNR
    snr = signal_power / noise_power
    # Convert to dB
    snr_db = 10 * np.log10(snr)
    return snr, snr_db

In [18]:
def generate_noisy_H(H_true, noise):
    # noise = np.random.normal(0, 10**(-5), H.shape)
    std = 10**(-4)
    if noise == gaussian:
        H_added_noise = gaussian(H_true, std)
    elif noise == 'pink':
        H_added_noise = np.zeros(H_true.shape)

    H_noise = H_true + H_added_noise

    return H_noise

In [19]:
def plot_heatmat(H):
    plt.figure(figsize=(8, 6))
    plt.imshow(H.T, cmap='viridis', aspect='auto', origin='lower', interpolation='nearest')
    plt.colorbar()
    plt.title('Channel Gain Magnitude of (RX, TX) Antenna Pairs')
    plt.xlabel('RX antenna')
    plt.ylabel('TX antenna')
    # plt.xticks(ticks=np.arange(H.shape[0]), labels=np.arange(1, H.shape[0] + 1))
    # plt.yticks(ticks=np.arange(H.shape[1]), labels=np.arange(1, H.shape[1] + 1))
    plt.show()

In [20]:
def data_generation(scenario_name, 
                    n_bs_y, n_bs_x, n_ue_y, n_ue_x,
                    n_paths, n_subcarriers):
    dataset = create_dataset(scenario_name, n_bs_y, n_bs_x, n_ue_y, n_ue_x, n_paths, n_subcarriers)
    H_true = generate_H_abs(dataset)
    # plot_environment(dataset)
    return H_true

Change the Environment Here

In [21]:
scenario_name = 'I2_28B'
n_bs_y, n_bs_x = 32, 1
n_ue_y, n_ue_x = 32, 1
n_paths = 5
n_subcarriers = 2

Change Parameters Here

In [22]:
n_datapoints = 100  # number of (total) datapoints generated
n_samples = 100

# Train/validation/test split parameters
train_size = 0.7
validation_size = 0.2
test_size = 0.1

noise = gaussian

if noise == pink:
    amplitude = 0.0001
    std = None
elif noise == gaussian:
    amplitude = None
    std = 10**(-6)

Generate & Save Datasets

In [31]:
scaler = MinMaxScaler()
H_true = data_generation(scenario_name,
                        n_bs_y, n_bs_x, n_ue_y, n_ue_x,
                        n_paths, n_subcarriers)

noisy_data = np.ndarray((n_samples, n_bs_y * n_ue_y))
for i in range(n_samples):
    if noise == pink:
        H_added_noise = pink(H_true, amplitude)
    elif noise == gaussian:
        H_added_noise = gaussian(H_true, std)

    H_noise = H_true + H_added_noise
    H_noise = scaler.fit_transform(H_noise)
    H_noise = H_noise.reshape(-1)
    noisy_data[i,:] = H_noise

H_true = scaler.fit_transform(H_true)
H_true = H_true.reshape(-1)

print(noisy_data.shape)
print(H_true.shape)


# create the datasets
true_data_df = pd.DataFrame([H_true] * noisy_data.shape[0])

noisy_data_df = pd.DataFrame(noisy_data)


# Split only the noisy data (X)
X_train, X_test, y_train, y_test = train_test_split(
    noisy_data_df,  
    true_data_df,
    test_size=0.2,   
    random_state=42  
)

# normalize the data




Basestation 1

UE-BS Channels


Reading ray-tracing: 100%|██████████| 402/402 [00:00<00:00, 90050.75it/s]


ToA of some paths of 199 channels with an average total power of 53.50% exceed the useful OFDM symbol duration and are clipped.


Generating channels: 100%|██████████| 201/201 [00:00<00:00, 5140.52it/s]



BS-BS Channels


Generating channels: 100%|██████████| 1/1 [00:00<00:00, 951.31it/s]

(100, 1024)
(1024,)


In [32]:
save = True
if save:
    folder_name = "../datasets/first_deepmimo_data"

    os.makedirs(folder_name, exist_ok=True)

    y_train.to_csv(f'{folder_name}/y_data_train.csv', index=False)
    y_test.to_csv(f'{folder_name}/y_data_test.csv', index=False)
    X_train.to_csv(f'{folder_name}/X_data_train.csv', index=False)
    X_test.to_csv(f'{folder_name}/X_data_test.csv', index=False)

